In [1]:
import re
import polars as pl
pl.Config.set_fmt_str_lengths(50)
print(pl.__version__)

from utils import xmp_to_taglist

0.20.13


In [2]:
col_taxon_file = "/home/wsyxbcl/Downloads/26a0a309-f729-42d3-8f76-56afa21686db/NameUsage.tsv"
col_name_file = "/home/wsyxbcl/Downloads/26a0a309-f729-42d3-8f76-56afa21686db/VernacularName.tsv"
shanshui_file = "/home/wsyxbcl/Downloads/_Species_Data__202401051603.csv"
gbif_backbone_file = "/home/wsyxbcl/Downloads/backbone/Taxon.tsv"
gbif_backbone_name_file = "/home/wsyxbcl/Downloads/backbone/VernacularName.tsv"

In [3]:
dtypes = {
    # "col:ID": pl.Categorical,
    "col:rank": pl.Categorical,
    "col:status": pl.Categorical,
    "col:combinationAuthorshipYear": pl.Utf8,
    "col:basionymAuthorshipYear": pl.Utf8,
}
# COL taxon data
df_col_taxon_raw = pl.read_csv(col_taxon_file, separator='\t', dtypes=dtypes, rechunk=True, quote_char=None)
df_col_taxon = df_col_taxon_raw.filter(pl.col("col:code").eq("zoological"))
df_col_taxon = df_col_taxon.select([
    pl.col("col:ID").alias("taxonID"), 
    pl.col("col:parentID").alias("parentID"), 
    pl.col("col:rank").alias("rank"), 
    pl.col("col:scientificName").str.replace(r' \([^)]*\)', '').alias("scientificName"), 
    pl.col("col:status").alias("status")])
# COL name data
df_col_name_raw = pl.read_csv(col_name_file, separator="\t")
df_col_name = df_col_name_raw.filter(pl.col("col:language") == "eng")
df_col_name = df_col_name.select([
    pl.col("col:taxonID").alias("taxonID"),
    pl.col("col:name").str.to_lowercase().alias("colVernacularNameEN"),
])
df_col_name = df_col_name.group_by("taxonID").agg([pl.col("colVernacularNameEN")])

df_col = df_col_taxon.join(df_col_name, on="taxonID", how="left")

In [4]:
# GBIF backbone data

df_gbif_taxon_raw = pl.read_csv(gbif_backbone_file, separator='\t', rechunk=True, quote_char=None)
# df_gbif_taxon = df_gbif_taxon_raw.filter(pl.col("taxonRank").is_in(["species", "subspecies"]))
df_gbif_taxon = (df_gbif_taxon_raw
    .filter(pl.col("kingdom").eq("Animalia"))
    .filter(pl.col("taxonRank").is_in(["species", "subspecies"]))
    .filter(pl.col("taxonomicStatus").eq("accepted"))
    .select([
        pl.col("taxonID").alias("gbifTaxonID"),
        pl.col("canonicalName").alias("scientificName"),
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus"
    ])
)

df_gbif_name_raw = pl.read_csv(gbif_backbone_name_file, separator="\t")
df_gbif_name = (df_gbif_name_raw
    # .filter((pl.col("language") == "en") | (pl.col("language").is_null()))
    .filter((pl.col("language") == "en") | (pl.col("source") == "NCBI Taxonomy"))
    .filter(pl.col("source") != "United Kingdom Species Inventory (UKSI)")
    .select([
        pl.col("taxonID").alias("gbifTaxonID"),
        pl.col("vernacularName").str.to_lowercase().alias("gbifVernacularNameEN"),])
    .group_by("gbifTaxonID")
    .agg([pl.col("gbifVernacularNameEN")])
)

df_gbif = df_gbif_taxon.join(df_gbif_name, on="gbifTaxonID", how="left")

In [21]:
# Taglists
taglist_yushu = xmp_to_taglist("/home/wsyxbcl/Downloads/玉树TagList.jpg.xmp")
taglist_qilian = xmp_to_taglist("/home/wsyxbcl/Downloads/祁连TagList.jpg.xmp")
taglist_lasa = xmp_to_taglist("/home/wsyxbcl/Downloads/拉萨TagList.jpg.xmp")
# taglist_yaluzangbu = xmp_to_taglist("/home/wsyxbcl/Downloads/雅鲁藏布TagList.jpg.xmp")

taglist = list((set(taglist_qilian) | set(taglist_yushu) | set(taglist_lasa)))
s_tag = pl.Series("tag", taglist).str.to_lowercase()

In [7]:
# # find unincluded tag in COL vernacular name
# tag_unincluded_in_col = []
# for tag in taglist:
#     try:
#         df_col.filter(pl.col("colVernacularNameEN").list.contains(tag.lower())).get_column("scientificName")[0]
#     except IndexError:
#         tag_unincluded_in_col.append(tag)
# print(tag_unincluded_in_col)

# # find unincluded tag in GBIF vernacular name
# tag_unincluded_in_gbif = []
# for tag in taglist:
#     try:
#         df_gbif.filter(pl.col("gbifVernacularNameEN").list.contains(tag.lower())).get_column("scientificName")[0]
#     except IndexError:
#         tag_unincluded_in_gbif.append(tag)
# print(tag_unincluded_in_gbif)

In [22]:
# Check tag in GBIF backbone
tag_list = []
latin_name_list = []
for tag in taglist:
    tag_list.append(tag)
    try:
        latin_name_list.append(df_gbif.filter(pl.col("gbifVernacularNameEN").list.contains(tag.lower())).get_column("scientificName").to_list())
    except IndexError:
        latin_name_list.append([])
    

In [24]:
latin_name_list

[['Felis catus'],
 ['Canis lupus dingo', 'Canis lupus familiaris'],
 ['Phylloscopus trochiloides'],
 ['Pyrrhocorax graculus'],
 ['Leptopoecile elegans'],
 ['Athene noctua'],
 ['Emberiza koslowi'],
 ['Naemorhedus baileyi', 'Nemorhaedus baileyi'],
 [],
 ['Aquila chrysaetos', 'Aquila bivia'],
 ['Phylloscopus fuscatus'],
 ['Parus minor'],
 ['Ochotona gloveri'],
 ['Carpodacus waltoni'],
 ['Trochalopteron elliotii'],
 [],
 [],
 ['Coloeus dauuricus'],
 ['Accipiter nisus'],
 ['Mycerobas carnipes'],
 ['Prunella fulvescens'],
 ['Tetraogallus himalayensis'],
 ['Martes foina'],
 ['Tetraophasis szechenyii'],
 ['Ochotona curzoniae'],
 ['Catopuma temminckii'],
 [],
 ['Carpodacus rubicilla'],
 [],
 ['Perdix hodgsoniae'],
 ['Phoenicurus frontalis'],
 ['Vulpes vulpes'],
 ['Corvus corax'],
 ['Carpodacus rubicilloides'],
 ['Mustela eversmanii'],
 [],
 ['Columba leuconota'],
 ['Montifringilla adamsi', 'Montifringilla henrici'],
 ['Lanius tephronotus'],
 ['Pyrgilauda ruficollis'],
 ['Moschus chrysogaster'],

In [25]:
# COL patch
latin_name_patched = []
for (i, names) in enumerate(latin_name_list):
    if len(names) > 1:
        # print(names)
        names_patched = []
        for name in names:
            if df_col.filter(pl.col("scientificName").eq(name)).is_empty():
                pass
            else:
                names_patched.append(name)
        latin_name_patched.append(names_patched)
    else:
        latin_name_patched.append(latin_name_list[i])

In [26]:
# output for edit
import csv
with open('taglist_to_be_edit.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(tag_list, ['|'.join(names) for names in latin_name_patched]))

In [97]:
# Manual patch

In [27]:
df_col.filter(pl.col("scientificName").eq("Chaimarrornis leucocephalus"))

taxonID,parentID,rank,scientificName,status,colVernacularNameEN
str,str,cat,str,cat,list[str]
"""TF6L""","""62M2T""","""species""","""Chaimarrornis leucocephalus""","""accepted""","[""white-capped redstart""]"


In [31]:
df_col.filter(pl.col("scientificName").eq("Panthera pardus"))

taxonID,parentID,rank,scientificName,status,colVernacularNameEN
str,str,cat,str,cat,list[str]
"""4CGXR""","""6DBT""","""species""","""Panthera pardus""","""accepted""","[""leopard""]"


In [108]:
df_col.filter(pl.col("scientificName").eq("Camelus ferus"))

taxonID,parentID,col:rank,scientificName,status,colVernacularNameEN
str,str,str,str,str,list[str]
"""Q9XF""","""Q9XD""","""species""","""Camelus ferus""","""ambiguous synonym""",null
"""5WWKW""","""7JFKC""","""species""","""Camelus ferus""","""ambiguous synonym""",null


In [32]:
df_col.filter(pl.col("taxonID").eq("4N9HG"))

taxonID,parentID,rank,scientificName,status,colVernacularNameEN
str,str,cat,str,cat,list[str]
"""4N9HG""","""6Y7B""","""species""","""Przewalskium albirostre""","""accepted""",null


In [64]:
df_col.filter(pl.col("scientificName").eq("Pica pica bottanensis"))

taxonID,parentID,rank,scientificName,status,colVernacularNameEN
str,str,cat,str,cat,list[str]
"""7KMF3""","""4HPXM""","""subspecies""","""Pica pica bottanensis""","""accepted""",null


In [38]:
df_col.filter(pl.col("scientificName").eq("Ursus arctos pruinosus"))

taxonID,parentID,rank,scientificName,status,colVernacularNameEN
str,str,cat,str,cat,list[str]
"""BP6VT""","""7F2KB""","""subspecies""","""Ursus arctos pruinosus""","""accepted""","[""brown bear""]"


In [41]:
df_taglist_patched = pl.read_csv("./taglist_sciname.csv", has_header=False).select([pl.col("column_1").alias("tag"), pl.col("column_2").alias("scientificName")])

In [42]:
df_taglist_patched

tag,scientificName
str,str
"""Eurasian sparrowhawk""","""Accipiter nisus"""
"""Chukar partridge""","""Alectoris chukar"""
"""Bar-headed goose""","""Anser indicus"""
"""Olive-backed pipit""","""Anthus hodgsoni"""
"""Golden eagle""","""Aquila chrysaetos"""
…,…
"""White-lipped deer""","""Przewalskium albirostre"""
"""Small rodents""","""Rodentia"""
"""Domestic camel""","""Camelus bactrianus"""


In [43]:
df_shanshui_raw = pl.read_csv(shanshui_file)
# Patch
iucn_patch_dict = {
    'LR/nt': 'NT',
    'LR/LC': 'LC'
}
rcb_patch_dict = {
    'Doubt:NT,VU,EN,CR': None
}
nationl_patch_dict = {
    '?': None,
    'Ⅰ': 'I'
}

df_shanshui_patch = df_shanshui_raw.select([
    pl.col("SpeciesID").alias("shanshuiTaxonID"),
    pl.col("ChineseName").alias("shanshuiVernacularNameCN"),
    pl.col("ScientificName").alias("scientificName"),
    pl.col("ClassCN").alias("shanshuiClassCN"),
    pl.col("OrderCN").alias("shanshuiOrderCN"),
    pl.col("FamilyCN").alias("shanshuiFamilyCN"),
    pl.col("GenusCN").alias("shanshuiGenusCN"),
    pl.col("IUCN").replace(iucn_patch_dict).alias("protectLevelIUCN"),
    pl.col("RCB").replace(rcb_patch_dict).alias("protectLevelRCB"),
    pl.col("NationalKey").replace(nationl_patch_dict).alias("protectLevelNationalKey"),
])

In [44]:
# Patch scientific name in shanshui data to COL
df_shanshui_col_patch = df_shanshui_patch.join(df_col, on="scientificName", how="left")

In [96]:
df_shanshui_col_patch.filter(pl.col("shanshuiVernacularNameCN") == "雪豹")

shanshuiTaxonID,shanshuiVernacularNameCN,scientificName,shanshuiClassCN,shanshuiOrderCN,shanshuiFamilyCN,shanshuiGenusCN,protectLevelIUCN,protectLevelRCB,protectLevelNationalKey,taxonID,parentID,rank,status,colVernacularNameEN
str,str,str,str,str,str,str,str,str,str,str,str,str,str,list[str]
"""MA0310""","""雪豹""","""Panthera uncia""","""哺乳纲""","""食肉目""","""猫科""","""豹属""","""VU""","""EN""","""I""","""4CGXT""","""7DHJ4""","""species""","""synonym""",null


In [59]:
df_shanshui_synonym = df_shanshui_col_patch.filter(pl.col("status").eq("synonym"))

synonym_parent_id = df_shanshui_synonym.get_column("parentID")
synonym_scientific_names = []
for id in synonym_parent_id:
    synonym_scientific_names.append(df_col.filter(pl.col("taxonID").eq(id)).get_column("scientificName")[0])

df_shanshui_synonym = df_shanshui_synonym.with_columns([
    pl.Series(synonym_scientific_names).alias("scientificName"),
    pl.lit("accepted").alias("status")])


In [57]:
df_shanshui_synonym

shanshuiTaxonID,shanshuiVernacularNameCN,scientificName,shanshuiClassCN,shanshuiOrderCN,shanshuiFamilyCN,shanshuiGenusCN,protectLevelIUCN,protectLevelRCB,protectLevelNationalKey,taxonID,parentID,rank,status,colVernacularNameEN
str,str,str,str,str,str,str,str,str,str,str,str,cat,cat,list[str]
"""AM0010""","""观雾小鲵""","""Hynobius fucus""","""两栖纲""","""有尾目""","""小鲵科""","""小鲵属""",null,"""EN""","""II""","""3NLTG""","""3NLTH""","""species""","""accepted""",null
"""AM0014""","""普雄原鲵""","""Pseudohynobius puxiongensis""","""两栖纲""","""有尾目""","""小鲵科""","""原鲵属""",null,"""CR""","""I""","""4N5MM""","""6WFRC""","""species""","""accepted""",null
"""AM0026""","""弱唇褶山溪鲵""","""Batrachuperus pinchonii""","""两栖纲""","""有尾目""","""小鲵科""","""山溪鲵属""",null,"""DD""","""II""","""KZSJ""","""KZSQ""","""species""","""accepted""",null
"""AM0035""","""丽色疣螈""","""Tylototriton shanjing""","""两栖纲""","""有尾目""","""蝾螈科""","""疣螈属""",null,"""NT""","""II""","""7DBM2""","""7DBLR""","""species""","""accepted""",null
"""AM0039""","""宽脊瑶螈""","""Tylototriton broadoridgus""","""两栖纲""","""有尾目""","""蝾螈科""","""瑶螈属""",null,"""NT""","""II""","""5CR6W""","""7DBLM""","""species""","""accepted""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""MA0284""","""小爪水獭""","""Amblonyx cinereus cinereus""","""哺乳纲""","""食肉目""","""鼬科""","""小爪水獭属""","""VU""","""EN""","""II""","""67L4L""","""5FJW6""","""species""","""accepted""",null
"""AV1216""","""西域山雀""","""Parus major bokharensis""","""鸟纲""","""雀形目""","""山雀科""","""山雀属""",null,"""LC""",null,"""75SVT""","""BNSLH""","""species""","""accepted""",null
"""MA0412""","""长吻真海豚""","""Delphinus delphis""","""哺乳纲""","""鲸目""","""海豚科""","""真海豚属""","""DD""","""DD""","""II""","""34JWC""","""34JWM""","""species""","""accepted""",null


In [98]:
df_shanshui_synonym.filter(pl.col("shanshuiVernacularNameCN") == "雪豹")

shanshuiTaxonID,shanshuiVernacularNameCN,scientificName,shanshuiClassCN,shanshuiOrderCN,shanshuiFamilyCN,shanshuiGenusCN,protectLevelIUCN,protectLevelRCB,protectLevelNationalKey,taxonID,parentID,rank,status,colVernacularNameEN
str,str,str,str,str,str,str,str,str,str,str,str,str,str,list[str]
"""MA0310""","""雪豹""","""Uncia uncia""","""哺乳纲""","""食肉目""","""猫科""","""豹属""","""VU""","""EN""","""I""","""4CGXT""","""7DHJ4""","""species""","""accepted""",null


In [47]:
df_shanshui_col_patch

shanshuiTaxonID,shanshuiVernacularNameCN,scientificName,shanshuiClassCN,shanshuiOrderCN,shanshuiFamilyCN,shanshuiGenusCN,protectLevelIUCN,protectLevelRCB,protectLevelNationalKey,taxonID,parentID,rank,status,colVernacularNameEN
str,str,str,str,str,str,str,str,str,str,str,str,cat,cat,list[str]
"""AM0009""","""台湾小鲵""","""Hynobius formosanus""","""两栖纲""","""有尾目""","""小鲵科""","""小鲵属""","""EN""","""EN""","""II""","""6N2LH""","""529J""","""species""","""accepted""",null
"""AM0012""","""楚南小鲵""","""Hynobius sonani""","""两栖纲""","""有尾目""","""小鲵科""","""小鲵属""","""EN""","""EN""","""II""","""6MQL8""","""529J""","""species""","""accepted""",null
"""AM0021""","""辽宁爪鲵""","""Onychodactylus zhaoermii""","""两栖纲""","""有尾目""","""小鲵科""","""爪鲵属""",null,"""CR""","""I""","""49R6N""","""68FG""","""species""","""accepted""",null
"""AM0013""","""商城肥鲵""","""Pachyhynobius shangchengensis""","""两栖纲""","""有尾目""","""小鲵科""","""肥鲵属""","""VU""","""VU""",null,"""75HVH""","""6BTW""","""species""","""accepted""",null
"""AM0022""","""吉林爪鲵""","""Onychodactylus zhangyapingi""","""两栖纲""","""有尾目""","""小鲵科""","""爪鲵属""","""DD""","""VU""","""II""","""49R6M""","""68FG""","""species""","""accepted""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""RE0018""","""凹甲陆龟""","""Manouria impressa""","""爬行纲""","""龟鳖目""","""陆龟科""","""凹甲陆龟属""","""EN""","""CR""","""I""","""3XW2F""","""5KYC""","""species""","""accepted""","[""impressed tortoise""]"
"""RE0018""","""凹甲陆龟""","""Manouria impressa""","""爬行纲""","""龟鳖目""","""陆龟科""","""凹甲陆龟属""","""EN""","""CR""","""I""","""88L6B""","""3XW2F""","""species""","""synonym""",null
"""RE0071""","""大壁虎""","""Gekko gecko""","""爬行纲""","""有鳞目""","""壁虎科""","""壁虎属""","""LC""","""CR""","""II""","""3FH5H""","""4MLL""","""species""","""accepted""","[""tokay gecko"", ""tokeh-tokeh"", ""tuctoo""]"


In [20]:
df_shanshui_synonym

shanshuiTaxonID,shanshuiVernacularNameCN,scientificName,shanshuiClassCN,shanshuiOrderCN,shanshuiFamilyCN,shanshuiGenusCN,protectLevelIUCN,protectLevelRCB,protectLevelNationalKey,taxonID,parentID,rank,status,colVernacularNameEN
str,str,str,str,str,str,str,str,str,str,str,str,cat,str,list[str]
"""AM0010""","""观雾小鲵""","""Hynobius fucus""","""两栖纲""","""有尾目""","""小鲵科""","""小鲵属""",null,"""EN""","""II""","""3NLTG""","""3NLTH""","""species""","""accepted""",null
"""AM0014""","""普雄原鲵""","""Pseudohynobius puxiongensis""","""两栖纲""","""有尾目""","""小鲵科""","""原鲵属""",null,"""CR""","""I""","""4N5MM""","""6WFRC""","""species""","""accepted""",null
"""AM0026""","""弱唇褶山溪鲵""","""Batrachuperus pinchonii""","""两栖纲""","""有尾目""","""小鲵科""","""山溪鲵属""",null,"""DD""","""II""","""KZSJ""","""KZSQ""","""species""","""accepted""",null
"""AM0035""","""丽色疣螈""","""Tylototriton shanjing""","""两栖纲""","""有尾目""","""蝾螈科""","""疣螈属""",null,"""NT""","""II""","""7DBM2""","""7DBLR""","""species""","""accepted""",null
"""AM0039""","""宽脊瑶螈""","""Tylototriton broadoridgus""","""两栖纲""","""有尾目""","""蝾螈科""","""瑶螈属""",null,"""NT""","""II""","""5CR6W""","""7DBLM""","""species""","""accepted""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""MA0284""","""小爪水獭""","""Amblonyx cinereus cinereus""","""哺乳纲""","""食肉目""","""鼬科""","""小爪水獭属""","""VU""","""EN""","""II""","""67L4L""","""5FJW6""","""species""","""accepted""",null
"""AV1216""","""西域山雀""","""Parus major bokharensis""","""鸟纲""","""雀形目""","""山雀科""","""山雀属""",null,"""LC""",null,"""75SVT""","""BNSLH""","""species""","""accepted""",null
"""MA0412""","""长吻真海豚""","""Delphinus delphis""","""哺乳纲""","""鲸目""","""海豚科""","""真海豚属""","""DD""","""DD""","""II""","""34JWC""","""34JWM""","""species""","""accepted""",null


In [60]:
df_shanshui_col_patch = df_shanshui_col_patch.with_columns(pl.col("status").cast(pl.Utf8))
df_shanshui = pl.concat([
    df_shanshui_col_patch.filter(pl.col("status").ne("synonym")),
    df_shanshui_synonym]).select([
        pl.col("shanshuiTaxonID"),
        pl.col("shanshuiVernacularNameCN"),
        pl.col("scientificName"),
        pl.col("shanshuiClassCN"),
        pl.col("shanshuiOrderCN"),
        pl.col("shanshuiFamilyCN"),
        pl.col("shanshuiGenusCN"),
        pl.col("protectLevelIUCN"),
        pl.col("protectLevelRCB"),
        pl.col("protectLevelNationalKey")
    ])
df_shanshui

shanshuiTaxonID,shanshuiVernacularNameCN,scientificName,shanshuiClassCN,shanshuiOrderCN,shanshuiFamilyCN,shanshuiGenusCN,protectLevelIUCN,protectLevelRCB,protectLevelNationalKey
str,str,str,str,str,str,str,str,str,str
"""AM0009""","""台湾小鲵""","""Hynobius formosanus""","""两栖纲""","""有尾目""","""小鲵科""","""小鲵属""","""EN""","""EN""","""II"""
"""AM0012""","""楚南小鲵""","""Hynobius sonani""","""两栖纲""","""有尾目""","""小鲵科""","""小鲵属""","""EN""","""EN""","""II"""
"""AM0021""","""辽宁爪鲵""","""Onychodactylus zhaoermii""","""两栖纲""","""有尾目""","""小鲵科""","""爪鲵属""",null,"""CR""","""I"""
"""AM0013""","""商城肥鲵""","""Pachyhynobius shangchengensis""","""两栖纲""","""有尾目""","""小鲵科""","""肥鲵属""","""VU""","""VU""",null
"""AM0022""","""吉林爪鲵""","""Onychodactylus zhangyapingi""","""两栖纲""","""有尾目""","""小鲵科""","""爪鲵属""","""DD""","""VU""","""II"""
…,…,…,…,…,…,…,…,…,…
"""MA0284""","""小爪水獭""","""Amblonyx cinereus cinereus""","""哺乳纲""","""食肉目""","""鼬科""","""小爪水獭属""","""VU""","""EN""","""II"""
"""AV1216""","""西域山雀""","""Parus major bokharensis""","""鸟纲""","""雀形目""","""山雀科""","""山雀属""",null,"""LC""",null
"""MA0412""","""长吻真海豚""","""Delphinus delphis""","""哺乳纲""","""鲸目""","""海豚科""","""真海豚属""","""DD""","""DD""","""II"""


In [100]:
df_shanshui.filter(pl.col("shanshuiVernacularNameCN") == "白唇鹿")

shanshuiTaxonID,shanshuiVernacularNameCN,scientificName,shanshuiClassCN,shanshuiOrderCN,shanshuiFamilyCN,shanshuiGenusCN,protectLevelIUCN,protectLevelRCB,protectLevelNationalKey
str,str,str,str,str,str,str,str,str,str
"""MA0349""","""白唇鹿""","""Przewalskium albirostre""","""哺乳纲""","""偶蹄目""","""鹿科""","""白唇鹿属""","""VU""","""EN""","""I"""


In [108]:
df_shanshui_raw.filter(pl.col("ChineseName") == "白唇鹿")

SpeciesID,ChineseName,ScientificName,ClassCN,Class,OrderCN,Order,FamilyCN,Family,GenusCN,Genus,SpeciesEpithet,SpeciesName,Subspecies,ChineseAlias,EnglishAlias,ScientificSynonym,Endemic,IUCN,IUCNCriteria,RCB,RCBCriteria,NationalKey,CITES,Notes
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""MA0349""","""白唇鹿""","""Przewalskium albirostris""","""哺乳纲""","""Mammalia""","""偶蹄目""","""ARTIODACTYLA""","""鹿科""","""Cervidae""","""白唇鹿属""","""Przewalskium""","""albirostris""",null,null,null,null,"""C. thorold (synonym);C. sellatus (synonym);Cervus…",null,"""VU""",null,"""EN""","""B1ab(i,ii,iii)+2ab(i,ii,iii)""","""I""",null,null


In [103]:
df_taglist_patched.filter(pl.col("tag") == "White-lipped deer")

tag,scientificName
str,str
"""White-lipped deer""","""Cervus albirostris"""


In [61]:
maze_taglist = df_taglist_patched.join(df_shanshui, on="scientificName", how="left")

In [62]:
maze_taglist_taxon = (maze_taglist
    .join(df_gbif.drop("gbifVernacularNameEN"), on="scientificName", how="left")
    .join(df_col.drop(["parentID", "status", "colVernacularNameEN"]), on="scientificName", how="left")
)

In [63]:
maze_taglist_taxon.write_csv("maze_taglist.csv")

In [35]:
df_col_taxon_raw.filter(pl.col("col:ID").eq("T569"))

col:ID,col:alternativeID,col:nameAlternativeID,col:sourceID,col:parentID,col:basionymID,col:status,col:scientificName,col:authorship,col:rank,col:notho,col:originalSpelling,col:uninomial,col:genericName,col:infragenericEpithet,col:specificEpithet,col:infraspecificEpithet,col:cultivarEpithet,col:combinationAuthorship,col:combinationAuthorshipID,col:combinationExAuthorship,col:combinationExAuthorshipID,col:combinationAuthorshipYear,col:basionymAuthorship,col:basionymAuthorshipID,col:basionymExAuthorship,col:basionymExAuthorshipID,col:basionymAuthorshipYear,col:namePhrase,col:nameReferenceID,col:publishedInYear,col:publishedInPage,col:publishedInPageLink,col:gender,col:genderAgreement,col:etymology,col:code,col:nameStatus,col:accordingToID,col:accordingToPage,col:accordingToPageLink,col:referenceID,col:scrutinizer,col:scrutinizerID,col:scrutinizerDate,col:extinct,col:temporalRangeStart,col:temporalRangeEnd,col:environment,col:species,col:section,col:subgenus,col:genus,col:subtribe,col:tribe,col:subfamily,col:family,col:superfamily,col:suborder,col:order,col:subclass,col:class,col:subphylum,col:phylum,col:kingdom,col:ordinal,col:branchLength,col:link,col:nameRemarks,col:remarks
str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""T569""",null,null,2144,"""4N9HG""",null,"""synonym""","""Cervus albirostris""","""Przewalski, 1883""","""species""",null,null,null,"""Cervus""",null,"""albirostris""",null,null,"""Przewalski""",null,null,null,"""1883""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""zoological""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""https://www.itis.gov/servlet/SingleRpt/SingleRpt?…",null,null


In [114]:
df_sh_bird_raw = pl.read_csv("/home/wsyxbcl/Downloads/sh_bird.csv").select(pl.col("物种名").alias("shanshuiVernacularNameCN"))

In [116]:
# df_sh_bird.join(df_shanshui_patch, on="shanshuiVernacularNameCN", how="left").filter(pl.col("shanshuiTaxonID").is_null())
df_sh_bird = df_sh_bird_raw.join(df_shanshui_col_patch, on="shanshuiVernacularNameCN", how="left").join(df_gbif, on="scientificName", how="left")

In [117]:
df_sh_bird

shanshuiVernacularNameCN,shanshuiTaxonID,scientificName,shanshuiClassCN,shanshuiOrderCN,shanshuiFamilyCN,shanshuiGenusCN,protectLevelIUCN,protectLevelRCB,protectLevelNationalKey,taxonID,parentID,rank,status,colVernacularNameEN,gbifTaxonID,kingdom,phylum,class,order,family,genus,gbifVernacularNameEN
str,str,str,str,str,str,str,str,str,str,str,str,str,str,list[str],i64,str,str,str,str,str,str,list[str]
"""暗绿绣眼鸟""","""AV1195""","""Zosterops japonicus""","""鸟纲""","""雀形目""","""绣眼鸟科""","""绣眼鸟属""","""LC""","""LC""","""三有""","""5D8MR""","""8B88""","""species""","""accepted""","[""japanese white-eye""]",9300456,"""Animalia""","""Chordata""","""Aves""","""Passeriformes""","""Zosteropidae""","""Zosterops""","[""japanese white-eye"", ""japanese white-eye (japanese)"", … ""japanese white-eye""]"
"""八哥""","""AV0734""","""Acridotheres cristatellus""","""鸟纲""","""雀形目""","""椋鸟科""","""八哥属""","""LC""","""LC""","""三有""","""9KHM""","""N9Y""","""species""","""accepted""","[""crested myna""]",2489010,"""Animalia""","""Chordata""","""Aves""","""Passeriformes""","""Sturnidae""","""Acridotheres""","[""crested myna"", ""crested myna"", … ""crested myna""]"
"""白腹鸫""","""AV0876""","""Turdus pallidus""","""鸟纲""","""雀形目""","""鸫科""","""鸫属""","""LC""","""LC""","""三有""","""59PMP""","""647VS""","""species""","""accepted""","[""pale thrush""]",2490773,"""Animalia""","""Chordata""","""Aves""","""Passeriformes""","""Turdidae""","""Turdus""","[""pale thrush"", ""pale thrush"", … ""pale thrush""]"
"""白腹蓝鹟""","""AV0909""","""Cyanoptila cyanomelana""","""鸟纲""","""雀形目""","""鹟科""","""Cyanoptila""","""LC""","""LC""","""三有""","""32NX9""","""3XPY""","""species""","""accepted""","[""blue-and-white flycatcher""]",2492391,"""Animalia""","""Chordata""","""Aves""","""Passeriformes""","""Muscicapidae""","""Cyanoptila""","[""blue-and-white flycatcher"", ""blue-and-white/zappey's flycatcher"", … ""blue-and-white flycatcher""]"
"""白骨顶""","""AV0288""","""Fulica atra""","""鸟纲""","""鹤形目""","""秧鸡科""","""骨顶属""","""LC""","""LC""","""三有""","""6K3W6""","""4LC2""","""species""","""accepted""","[""eurasian coot""]",2474377,"""Animalia""","""Chordata""","""Aves""","""Gruiformes""","""Rallidae""","""Fulica""","[""eurasian coot"", ""eurasian coot"", … ""common coot""]"
"""白鹡鸰​""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""白鹭""","""AV0049""","""Egretta garzetta""","""鸟纲""","""鹈形目""","""鹭科""","""白鹭属""","""LC""","""LC""","""三有""","""6DYRB""","""62QNG""","""species""","""accepted""","[""little egret""]",2480876,"""Animalia""","""Chordata""","""Aves""","""Pelecaniformes""","""Ardeidae""","""Egretta""","[""little egret/western reef-heron"", ""little egret"", … ""little egret""]"
"""白眉鸫""","""AV0875""","""Turdus obscurus""","""鸟纲""","""雀形目""","""鸫科""","""鸫属""","""LC""","""LC""","""三有""","""59PMJ""","""647VS""","""species""","""accepted""","[""eyebrowed thrush""]",2490728,"""Animalia""","""Chordata""","""Aves""","""Passeriformes""","""Turdidae""","""Turdus""","[""eyebrowed thrush"", ""eyebrowed thrush"", … ""grey-headed thrush""]"
"""白眉鹀""","""AV1355""","""Emberiza tristrami""","""鸟纲""","""雀形目""","""鹀科""","""鹀属""","""LC""","""NT""","""三有""","""39FTW""","""4B69""","""species""","""accepted""","[""tristram's bunting""]",2491608,"""Animalia""","""Chordata""","""Aves""","""Passeriformes""","""Emberizidae""","""Emberiza""","[""tristram's bunting"", ""tristram's bunting"", … ""tristram's bunting""]"


In [130]:
df_sh_bird.with_columns([
    (pl.col("gbifVernacularNameEN").list.unique().list.join(", ")).alias("gbifVernacularNameEN"),
    (pl.col("colVernacularNameEN").list.unique().list.join(", ")).alias("colVernacularNameEN"),
]).drop(["parentID", "status"]).rename({"shanshuiVernacularNameCN": "vernacularNameCN"}).write_csv("./maze_taxonomy_jinhai.csv")


In [124]:
df_shanshui_raw.filter(pl.col("SpeciesID") == "AV0048")

SpeciesID,ChineseName,ScientificName,ClassCN,Class,OrderCN,Order,FamilyCN,Family,GenusCN,Genus,SpeciesEpithet,SpeciesName,Subspecies,ChineseAlias,EnglishAlias,ScientificSynonym,Endemic,IUCN,IUCNCriteria,RCB,RCBCriteria,NationalKey,CITES,Notes
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""AV0048""","""中白鹭""","""Ardea intermedia""","""鸟纲""","""Aves""","""鹈形目""","""Pelecaniformes""","""鹭科""","""Ardeidae""","""鹭属""","""Ardea""","""intermedia""",null,null,null,null,"""Egretta intermedia (synonym)""",null,"""LC""",null,"""LC""",null,"""三有""",null,null


In [126]:
df_col.filter(pl.col("scientificName") == "Ardea intermedia")

taxonID,parentID,rank,scientificName,status,colVernacularNameEN
str,str,str,str,str,list[str]


In [6]:
df_colchina_raw = pl.read_excel("/home/wsyxbcl/Downloads/动物界-脊索动物门-2023-8476.xlsx")

In [8]:
df_colchina_raw.filter(pl.col("种中文名").eq("白唇鹿"))

界中文名,界拉丁名,门中文名,门拉丁名,纲中文名,纲拉丁名,目中文名,目拉丁名,科中文名,科拉丁名,属中文名,属拉丁名,种中文名,种拉丁名,审核专家,名录版本
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64
"""动物界""","""Animalia""","""脊索动物门""","""Chordata""","""哺乳纲""","""Mammalia""","""鲸偶蹄目""","""CETARTIODACTYLA""","""鹿科""","""Cervidae""","""白唇鹿属""","""Przewalskium""","""白唇鹿""","""Przewalskium albirostris""","""Wei FuWen、Hu YiBo""",2023


In [ ]:
# Check scientific name difference 
df_colchina = df_colchina_raw.select([
    pl.col("种拉丁名").alias("scientificName")
])
df_colchina_compare = df_colchina.join(df_col, on="scientificName", how="left")

taxonID,parentID,rank,scientificName,status,colVernacularNameEN
str,str,str,str,str,list[str]
"""57DH4""","""7XMV""","""species""","""Tomomyza nitidula""","""accepted""",null
"""92VBG""","""8C85H""","""species""","""Oncopsia dubia""","""synonym""",null
"""XYRB""","""7NTGP""","""species""","""Convoluta variabilis""","""accepted""",null
"""7SPV9""","""7QH3R""","""species""","""Physa permembranacea""","""synonym""",null
"""37NKH""","""48BD""","""species""","""Drilaster shibatai""","""accepted""",null
"""GSS3""","""7NPQ8""","""species""","""Armina cordellensis""","""accepted""",null
"""96MQH""","""927N2""","""species""","""Dyspessa zurvan""","""accepted""",null
"""BH5S2""","""7QFSG""","""species""","""Helix dentrita""","""synonym""",null
"""928Q2""","""92HJP""","""genus""","""Euctenurapteryx""","""synonym""",null
